# Exploratory data analysis

## Manual exploration of data

In [ ]:
import pandas as pd
from pathlib import Path

# Simple relative path from root directory
FILEPATH = Path("data/raw/CPT_PremstallerGeotechnik_revised.csv")
df = pd.read_csv(FILEPATH)
df.head()


In [ ]:
df.columns

In [ ]:
df.info()

Features defined by Rauter and Tschuchnigg (2019). Note: several features are derived feature engineered features.
- Derived from other features: 'Rf (%)', 'Qt (-)', 'Qtn (-)', 'Fr (%)'
- Calculated: 'γ (kN/m³)', 'σ,v (kPa)',
       'u0 (kPa)', 'σ',v (kPa)', 'Depth (m)'
- Measured: 'qc (MPa)', 'fs (kPa)'

In [ ]:
FEATURES = [
    "Depth (m)",
    "qc (MPa)",
    "fs (kPa)",
    "Rf (%)",
    "σ,v (kPa)",
    "u0 (kPa)",
    "σ',v (kPa)",
    "Qtn (-)",
    "Fr (%)",
]
SITE_INFO = ["ID", "test_type", "basin_valley"]
LABELS_O = ["Oberhollenzer_classes"]
df = df.loc[:, SITE_INFO + FEATURES + LABELS_O]
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
df = df.dropna()
df.head()

In [ ]:
df.shape

The balance of the target label

In [ ]:
df["Oberhollenzer_classes"].value_counts()

In [ ]:
# remove class 0 since that is a bagging class
df = df[df["Oberhollenzer_classes"] != 0]
df.shape

In [ ]:
# map the class numbers to more descriptive names using the Oberhollenzer classification
class_mapping = {
    1: "Gravel",
    2: "Fine grained organic soils",
    3: "Coarse grained organic soils",
    4: "Sand to gravel",
    5: "Sand",
    6: "Silt to fine sand",
    7: "Clay to silt",
}
# visualise value counts per class
df["Oberhollenzer_classes"] = df["Oberhollenzer_classes"].map(class_mapping)

In [ ]:
df["basin_valley"].value_counts()

In [ ]:
# run describe on the numberical features
pd.set_option("display.float_format", "{:.2f}".format)
numeric_features = df[FEATURES].select_dtypes(include=["number"]).columns
df[numeric_features].describe()
# pd.reset_option('display.float_format')

## Automated EDA

ydata-profiling for automated EDA

In [ ]:
dfy = df.copy()[FEATURES + LABELS_O]
dfy.head()

In [ ]:
dfy.info()

In [ ]:
from ydata_profiling import ProfileReport

profile = ProfileReport(dfy, title="Profiling report")
profile.to_file("dataset_profiling_cpt.html")

In [ ]:
profile.to_notebook_iframe()

What do you need to check in the dataset?
- What distributions do the features have?
- Are there any missing values?
- Are there any duplicates?
- Are there any outliers? Be cautious with this one, as outliers can be valid data points.
- Are there any relationships between the features? Some features might be removed if they are highly correlated.
- Are there any relationships between the features and the target variable? Only in regression.
- Are there any relationships between the features and the target variable that are not linear? Only in regression.
- Are the labels balanced?


Trends in the data
- Not normally distributed
- Duplicate values
- Correlated features
- outliers present

## Preprocessing

In [ ]:
df.shape

Duplicate values

In [ ]:
df = df.drop_duplicates(subset=FEATURES)
df.shape

### Outliers

Hardcoded values (values that are not possible in the real world) are removed.

In [ ]:
# hardcoded
df = df[df["qc (MPa)"] > 0]
df = df[df["u0 (kPa)"] >= 0]
df = df[df["Qtn (-)"] > 0]
df = df[(df["fs (kPa)"] < 1200) & (df["fs (kPa)"] > 0)]
# df = df[(df['Rf (%)'] < 10) & (df['Rf (%)'] > 0)]
df = df[df["Rf (%)"] > 0]
df = df[(df["Fr (%)"] < 10) & (df["Fr (%)"] > 0)]
# skip samples with label 3.0
# df = df[df['Oberhollenzer_classes'] != 3.0] # due to low sample size
# df = df[df["Oberhollenzer_classes"] != 0.0]  # due to low sample size
df.shape

Univariate outlier detection - we exemplify for one feature

In [ ]:
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt

df["Qtn (-)"].hist(bins=100)
plt.show()

In [ ]:
# univariate
from pyod.models.mad import MAD

threshold = 7.0  # Typical value, can be adjusted for sensitivity
mad = MAD(threshold=threshold)

# Fit the model on the column
mad.fit(df[["Qtn (-)"]])

# Predict outliers (1 for outlier, 0 for inlier)
outliers = mad.predict(df[["Qtn (-)"]])

# Filter the DataFrame to exclude outliers
df_no_outliers = df[outliers == 0]

In [ ]:
sum(outliers)

In [ ]:
df_no_outliers["Qtn (-)"].hist(bins=100)

Multivariate outlier detection

In [ ]:
df[FEATURES].info()

In [ ]:
from pyod.models.iforest import IForest

outlier_confidence_threshold = 0.95  # Adjust threshold for your use case

# Initialize and fit the Isolation Forest model
iforest = IForest(n_estimators=100)
iforest.fit(df[FEATURES])

# Get the outlier probabilities
probs = iforest.predict_proba(df[FEATURES])[:, 1]

# Create a mask for outliers based on the confidence threshold
is_outlier = probs > outlier_confidence_threshold
outliers = df[is_outlier]
non_outliers = df[~is_outlier]

# Display results
num_outliers = len(outliers)
print(f"Number of outliers with Isolation Forest: {num_outliers}")
print(f"Percentage of outliers: {num_outliers / len(df):.4f}")
print("Outlier samples:\n", outliers)

# Cleaned DataFrame excluding outliers
df_cleaned = df[~is_outlier]

A classic splitting of the dataset in train and test sets

In [ ]:
from sklearn.model_selection import train_test_split

X = df_cleaned[FEATURES]
y = df_cleaned["Oberhollenzer_classes"]
seed = 10
test_size = 0.25
train, test = train_test_split(
    df_cleaned, test_size=test_size, random_state=seed, stratify=y
)

In [ ]:
train.shape, test.shape

Inspecting the effect of stratification

In [ ]:
train["Oberhollenzer_classes"].value_counts(normalize=True).round(2)

In [ ]:
test["Oberhollenzer_classes"].value_counts(normalize=True).round(2)

Typical geotechnical problem: data from same borehole should not be split between train and test set. Then there is a risk of data leakage.

In [ ]:
unique_ids = df["ID"].unique()
train_ids, _ = train_test_split(unique_ids, test_size=test_size, random_state=seed)
train_ids

In [ ]:
train_df = df[df["ID"].isin(train_ids)]
test_df = df[~df["ID"].isin(train_ids)]

In [ ]:
train_df.shape, test_df.shape

In [ ]:
train_df["Oberhollenzer_classes"].value_counts(normalize=True).round(2)

In [ ]:
test_df["Oberhollenzer_classes"].value_counts(normalize=True).round(2)

In [ ]:
# another round in ydate-profiling with splitted dataset
from ydata_profiling import ProfileReport

dfy_train = train_df.copy()[FEATURES + LABELS_O]
dfy_test = test_df.copy()[FEATURES + LABELS_O]
# dfy_train["Oberhollenzer_classes"] = dfy_train["Oberhollenzer_classes"].astype(str)
# dfy_test["Oberhollenzer_classes"] = dfy_test["Oberhollenzer_classes"].astype(str)
profile_train = ProfileReport(dfy_train, title="Profiling report - Train")
profile_test = ProfileReport(dfy_test, title="Profiling report - Test")
comparison_report = profile_train.compare(profile_test)
comparison_report.to_file("dataset_profiling_comparison_cpt.html")
comparison_report.to_notebook_iframe()


## Save preprocessed data

Save the train and test sets to CSV files for use in model training notebooks.

In [ ]:

# Simple relative path from root directory
output_dir = Path("data/model_ready")
output_dir.mkdir(parents=True, exist_ok=True)

# Save train and test datasets
train_df.to_csv(output_dir / "dataset_train.csv", index=False)
test_df.to_csv(output_dir / "dataset_test.csv", index=False)

print(f"Train dataset saved: {train_df.shape[0]} rows, {train_df.shape[1]} columns")
print(f"Test dataset saved: {test_df.shape[0]} rows, {test_df.shape[1]} columns")
print(f"Files saved to: {output_dir.resolve()}")
